<a href="https://colab.research.google.com/github/pushyag1/BigData_PySpark/blob/main/Pyspark_UpdatedCompiled.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title PySpark Setup(run me!)
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-3.0.2/spark-3.0.2-bin-hadoop2.7.tgz
!tar xf spark-3.0.2-bin-hadoop2.7.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.2-bin-hadoop2.7"

import findspark
findspark.init()

findspark.find()

from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

spark



Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:14 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:15 http://ppa.launchpad

In [269]:
#@title Dynamic Input(select your choices and run the cell)
Date_to_calculate_age = '2021-02-01' #@param {type:"date"}
Choose_a_model_version = 'ESRD-P1' #@param ["V22","V24", "RX", "ESRD-P1","ESRD-P2"]
Choose_a_model_year = '2021' #@param ["2020","2021"]
Sex_Age_edits_required = 'Yes' #@param ["Yes","No"]

In [303]:
#@title Age Calculation
import pandas 
import os
import pyspark.sql.functions as func
from pyspark.sql.functions import datediff, to_date, lit
from pyspark.sql.functions import struct
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import udf


#Loading Persons file
df1 = pandas.read_excel ("/content/Person-file.xlsx")
#df1=df.drop(['Unnamed: 8', 'Unnamed: 9','Unnamed: 10'], axis=1)
df1 = spark.createDataFrame(df1)

df2=df1.withColumn("Age",(datediff(to_date(lit(Date_to_calculate_age)),to_date("DOB","yyyy/MM/dd")))/366)
df3 = df2.withColumn("Age", func.round(df2["Age"], 2).cast('integer'))

#Condition for AGE and OREC
def func(Age, OREC):
  if Age == 64 and OREC == 0:
    return 65
  elif Age < 0 :
    return 0
  return Age


func_udf = udf(func, IntegerType())
df4 = df3.withColumn('new_column',func_udf(df3['Age'], df3['OREC']))

drop_list = [ 'Age',]
sdf5=df4.select([column for column in df4.columns if column not in drop_list])

df_final = sdf5.withColumnRenamed("new_column", "Age")

df_final.show()


+---------+-------------------+------+----+--------+-------+----+-----+---+
|Person ID|                DOB|Gender|OREC|LTIMCAID|NEMCAID|ESRD|MCAID|Age|
+---------+-------------------+------+----+--------+-------+----+-----+---+
| 54891178|1946-09-04 00:00:00|     2|   3|       1|      1|   0|    0| 74|
|   227107|1952-07-29 00:00:00|     2|   2|       0|      0|   1|    1| 68|
|  5155049|1950-08-11 00:00:00|     2|   0|       0|      0|   1|    1| 70|
|  5766696|1948-05-06 00:00:00|     2|   3|       0|      0|   0|    0| 72|
|   324986|1947-12-18 00:00:00|     2|   3|       0|      0|   1|    0| 72|
|  4480355|1947-02-17 00:00:00|     1|   1|       0|      0|   0|    0| 73|
|   779521|1949-04-28 00:00:00|     1|   1|       0|      0|   0|    0| 71|
|   302256|1940-01-23 00:00:00|     2|   1|       0|      1|   0|    0| 80|
|  8053184|1945-10-07 00:00:00|     1|   0|       0|      1|   0|    0| 75|
| 55688818|1954-10-07 00:00:00|     1|   0|       0|      0|   0|    0| 66|
| 62697035|1

###Please Refer above df_final(dataframe for further calculation)-Himanshu

In [ ]:
#@title Demography Variable

import pandas as pd
from pyspark.sql.functions import datediff, to_date, lit
from pyspark.sql.functions import col, expr, when
import pyspark.sql.functions as F
from pyspark.sql.types import *

df_age = df_final.toPandas()
df_final_copy = df_age.copy()
data = spark.createDataFrame(df_final_copy)

demo = (spark.read.format("csv").options(header="true").load("/content/Demography Variable Calculations.csv"))

demo = demo.filter((demo['Version'] == Choose_a_model_version) & (demo['Year'] == Choose_a_model_year))
data = data.withColumn("Year", lit(str(Choose_a_model_year)))
data = data.withColumn("Version", lit(str(Choose_a_model_version)))

for i in demo.collect():
  data = data.withColumn(i[2], expr(i[3]))

data.show()

#data.toPandas().to_excel('v24'+ '.xlsx')

In [ ]:
#@title CC_Mapping

import pandas as pd
import numpy as np
import time
from pyspark.sql.functions import length,col,trim
from pyspark.sql.functions import UserDefinedFunction
from pyspark.sql.types import *
from pyspark.sql import functions as F
from pyspark.sql.functions import col

df_diagnosis = pd.read_csv('Diagnosis Input File.csv')
df_diagnosis = spark.createDataFrame(df_diagnosis)
df_diagnosis = df_diagnosis.withColumn('DIAGNOSIS CODE',trim(col("DIAGNOSIS CODE")))

df_CC_mapping = pd.read_csv('CC Mapping.csv')
label_schema = StructType([
    StructField("YEAR", StringType()),
    StructField("VERSION", StringType()),
    StructField("DIAGNOSIS CODE", StringType()),
    StructField("CC", IntegerType())
])
df_CC_mapping = spark.createDataFrame(df_CC_mapping,schema= label_schema)


start=time.time()
# for year in model_years:
#   for version in versions:
cond1= col('YEAR')== str(Choose_a_model_year)
cond2= col('VERSION')== str(Choose_a_model_version)
df_temp= df_CC_mapping.where(cond1 & cond2)

# df_temp.show(5)
df_temp= df_temp.toDF('YEAR','VERSION','D_DIAG CODE','CC')

h= df_temp.join(df_diagnosis,(df_diagnosis["DIAGNOSIS CODE"] == df_temp["D_DIAG CODE"]),how='right')  # Mtachin diagnosis codes from diagnosis input file and CC_mapping file

unique_HCC= sorted([i.CC for i in df_temp.select('CC').distinct().collect()]) #Finding all unique values of HCC and storing it in a list to generate HCC columns later

CC_list = h.select("CC").rdd.flatMap(lambda x: x).collect()

p_id = h.select("PERSON ID").rdd.flatMap(lambda x: x).collect()

diag_code= h.select("DIAGNOSIS CODE").rdd.flatMap(lambda x: x).collect()


df_main= pd.DataFrame()
df_main['PERSON ID']= p_id
df_main['DIAGNOSIS CODE']= diag_code 

df_main['HCC']= CC_list  # CC values obtained after mapping from Part 2.
df_main['HCC'] = list(df_main['HCC'].astype(np.float).astype("Int32"))

df_main['MODEL YEAR']= str(Choose_a_model_year)
df_main['Version']=str(Choose_a_model_version)

# df_main
if 'RX' in Choose_a_model_version:
  columns_df=[]
  for value in (unique_HCC):
    columns_df.append('RXHCC'+str(value)) # Example: RXHCC1,RXHCC2,RXHCC3 etc
  #print(columns_df)
  for colum in range(len(columns_df)):
    df_main[columns_df[colum]]=0  #adding zero's to every column 

  for v in range(len(df_main)):                         
    df_main.at[v,'RXHCC'+ str(df_main.iloc[v][2])]=1   # Adding 1's to columns. Example if df_main['HCC'][10]= 22, then adding a 1 in HCC10 column in the 9th row. 

  df_main = df_main.drop(df_main.columns[-1],axis=1)
  #df_main.to_excel(str(Choose_a_model_version)+'_'+str(Choose_a_model_year)+'.xlsx')


else:
  # Part 5: Creating new columns with column names as HCC values obtained from Part 4
  columns_df=[]
  for value in (unique_HCC):
    columns_df.append('HCC'+str(value)) # Example: HCC1,HCC2,HCC3 etc
  #print(columns_df)
  for colum in range(len(columns_df)):
    df_main[columns_df[colum]]=0  #adding zero's to every column 

  for v in range(len(df_main)):                         
    df_main.at[v,'HCC'+ str(df_main.iloc[v][2])]=1   # Adding 1's to columns. Example if df_main['HCC'][10]= 22, then adding a 1 in HCC10 column in the 9th row. 

  df_main = df_main.drop(df_main.columns[-1],axis=1)
  #df_main.to_excel(str(Choose_a_model_version)+'_'+str(Choose_a_model_year)+'.xlsx')

print(time.time()-start)
df_main

## @CC-Override

In [ ]:
#@title CC-Override
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
type(df_main)

In [ ]:
type(data)

In [ ]:
data = data.toPandas()


In [ ]:
type(data)

In [ ]:
data.rename(columns = {'Person ID':'PERSON ID'}, inplace = True) 

In [ ]:
df = pd.merge(df_main,data,on ="PERSON ID", how="inner")
df

In [ ]:
df=df.drop(columns=['Year','Version_y'])

In [ ]:
df.rename(columns={'Version_x':'Version'}, inplace = True)

In [ ]:
type(df)

In [ ]:
df.rename(columns={'DIAGNOSIS CODE':'DIAG', 'GENDER':'SEX', 'Age':'AGEF'}, inplace = True)

In [ ]:
versions = df['Version'].unique().tolist()
versions

In [ ]:
years = df['MODEL YEAR'].unique().tolist()
years

In [ ]:
df.dtypes

In [ ]:
df.columns

In [ ]:
df.to_excel('Merged_df1.xlsx')

In [ ]:
data1 = pd.read_excel('/content/Merged_df1.xlsx')

In [ ]:
df1 = data1.copy()
data = spark.createDataFrame(df1)
    
overide_df = (spark.read.format("csv").options(header="true").load("/content/drive/MyDrive/Index/CC Override Rules.csv"))
overide_df = overide_df.withColumnRenamed("MODEL YEAR","MODEL_YEAR")
overide_df = overide_df.filter((overide_df.VERSION.isin(versions))&(overide_df.MODEL_YEAR.isin(years)))
for i in overide_df.collect():
  data = data.withColumn('HCC', expr(i[3]))
data.toPandas().to_excel('Merged_CCOveride.xlsx')  
    
#Merged file for CCOverride

In [ ]:
df_CCmerged = pd.read_excel('/content/Merged_CCOveride.xlsx')
df_CCmerged

In [ ]:
df_CCmerged=df_CCmerged.drop(columns=['Unnamed: 0',	'Unnamed: 0.1'])

In [ ]:
dRX=df_CCmerged[df['Version']== 'RX']
dRX = dRX.reset_index(drop=True)
dRX

In [ ]:
Versions=df_CCmerged['Version'].unique()
Versions
years=df_CCmerged['MODEL YEAR'].unique()

In [ ]:
Versions

In [ ]:
result1=[]
for version, df_version in df_CCmerged.groupby('Version'):
  for year, df_year in df_version.groupby('MODEL YEAR'):
    result1.append(df_year)

result = []
for dat in result1:
  dat = result.append(dat.groupby(['PERSON ID'] ).max().reset_index())

In [ ]:
[d1]=result


In [ ]:
final = pd.concat(result)

In [ ]:
final.head(2)

In [ ]:
final.to_excel('Merged_rowdf.xlsx') #merged data 

In [ ]:

#Files for single row for single personID for each year and version

In [ ]:
d1.to_excel('ESRD-P1-2020.xlsx')
d2.to_excel('ESRD-P1-2021.xlsx')
d3.to_excel('ESRD-P2-2020.xlsx')
d4.to_excel('ESRD-P2-2021.xlsx')
d5.to_excel('RX-2020.xlsx')
d6.to_excel('RX-2021.xlsx')
d7.to_excel('V22-2020.xlsx')
d8.to_excel('V22-2021.xlsx')
d9.to_excel('V24-2020.xlsx')
d10.to_excel('V24-2021.xlsx')

-- END OF CC-OVERRIDE with single row for each person ID for for each year and version

HCC HIERARCHY STARTS HERE

In [265]:
data = pd.read_excel('/content/gdrive/My Drive/HCC Hierarchy (1).xlsx')   #Loading Files
data1 = spark.createDataFrame(data)                                       #Making Spark Dataframe
data1.show()                                                              #Making Spark Dataframe
merged_cc = pd.read_excel('/content/gdrive/My Drive/Merged_rowdf.xlsx')  #Loading File Created by HCC Hierarchy Team

versions = data['MODEL_VRSN'].unique()                                    #Taking unique values to iterate later
years = data['MODEL_YR'].unique()                                         #Taking unique values to iterate later

from pyspark.sql import functions as F


#Section to create hierarchy table for all unique years and version being passed from above. This ensure it to be dynamic.
lis = {}
for k in range(len(years)):
  for l in range(len(versions)):
    var = str(years[k]) + '_' +  str(versions[l])
    lis[var] = []
    data_2020_test = data[(data['MODEL_YR'] == years[k]) & (data['MODEL_VRSN'] == versions[l])]
    merged_cc_2020_test = merged_cc[(merged_cc['MODEL YEAR'] == years[k]) & (merged_cc['Version'] == versions[l]) ]
    merged_cc_2020_spark_test = spark.createDataFrame(merged_cc_2020_test)
    for i in range(len(data_2020_test)):
      merged_cc_2020_spark1 = 0
      if list(data_2020_test['HIGHER_HCC'])[i] in merged_cc_2020_spark_test.columns:
        merged_cc_2020_spark1 = merged_cc_2020_spark_test.withColumn(list(data_2020_test['LOWER_HCC'])[i], F.when((F.col(list(data_2020_test['HIGHER_HCC'])[i])==1) & (F.col(list(data_2020_test['LOWER_HCC'])[i])== 1) & (F.col('VERSION')== list(data_2020_test['MODEL_VRSN'])[i]) ,0).otherwise(F.col(list(data_2020_test['LOWER_HCC'])[i])))
        # print(i)
    lis[var].append(merged_cc_2020_spark1)



# Creating seperate files for different years and versions and saving excel files.
mer_frames = []
for i in lis.keys():
  print(i)
  if lis[i][0]!=0:  
    frame = lis[i][0]
    frame = frame.toPandas()
    mer_frames.append(frame)
    frame.to_excel('/content/gdrive/My Drive/' + str(i) + '.xlsx')

#Creating merged database to be passed to the interaction variables calculation.
result = pd.concat(mer_frames)

#Savin
result.to_excel('/content/gdrive/My Drive/Merged_file_for_Akarshika_updated.xlsx')




FileNotFoundError: ignored

END OF HCC HIERARCHY



Interaction Variable Calculation



In [ ]:
from pyspark.sql.functions import datediff, to_date, lit
from pyspark.sql.functions import col, expr, when
import pyspark.sql.functions as F
from pyspark.sql.types import *

result= pd.read_csv('Merged_file_for_HCC_updated.csv')
df1_copy = result.copy()
data_interaction = spark.createDataFrame(df1_copy)

interaction_demo = (spark.read.format("csv").options(header="true").load("/content/Interaction Variable Calculations - 1.csv"))

interaction_demo = interaction_demo.filter((interaction_demo['MODEL VRSN'] == Choose_a_model_version) & (interaction_demo['MODEL YR'] == Choose_a_model_year))
data_interaction = data_interaction.where(F.col('MODEL YEAR') == Choose_a_model_year)
data_interaction = data_interaction.withColumn("Version", lit(str(Choose_a_model_version)))

for i in interaction_demo.collect():
  data_interaction = data_interaction.withColumn(i[2], expr(i[3]))

data_interaction.toPandas().to_excel('Merged_Interaction_Variable.xlsx') 


End of Interaction Variable Calculation